<a href="https://colab.research.google.com/github/dcovarru/Python/blob/master/Application_Crytpo_Currency_Create_Daily_Dev_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages


In [ ]:
########################
# Load packages 
########################

import pandas as pd
import datetime
import numpy as np
import pandas_datareader
import pandas_datareader.data as web
from pandas_datareader.data import get_quote_yahoo
import matplotlib.pyplot as plt
import os as os

from google.colab import drive
from google.cloud import bigquery

import statsmodels.api as sm
from statsmodels.tsa.api import VAR

from datetime import datetime, timedelta, date


# from pandas import Series, DataFrame
# from pandas.plotting import scatter_matrix
# import os
# import re
# import plotly.figure_factory as ff
# import plotly as py
# import plotly.offline as offline
# import plotly.graph_objects as go
# import plotly.express as px
# from google.colab import drive
# import io

import seaborn as sns
%matplotlib inline

# import sys
# print(sys.path)

# Install Yahoo Finance

In [ ]:
!pip install yfinance

import yfinance as yf

# Import Coinbase Tickers

In [ ]:
##############################################
# Use this code to read and write coins to DB from Google Sheets
##############################################

from google.colab import auth
auth.authenticate_user()
print('Google Authenticated')

# ##############################################
# # read in historical data previously created #
# ##############################################

# drive.mount('/content/drive')
# coinsDF = pd.read_excel("drive/My Drive/Data/coinbase_coins.xlsx")
# drive.flush_and_unmount()

# ############################
# # Write to BigQuery
# ############################

# table_id = 'my_dataset.coinbase_coins'
# coinsDF.to_gbq(table_id, project_id = 'sports-betting1', if_exists = 'replace')


Google Authenticated


In [ ]:
##############################################
# read in coins from DB
##############################################

%%bigquery --project sports-betting1 coinbase_coins

SELECT 
  * 
FROM `my_dataset.coinbase_coins`
--order by Coin asc

In [ ]:
coinbase_coins.iloc[0,0]


'1INCH'

In [ ]:
coin = []

In [ ]:
coin

[]

In [ ]:
coinbase_coins.shape

(66, 1)

In [ ]:
coinbase_coins.iloc[1, 0]

'AAVE'

# Grab Data

In [ ]:
dfOut = []
dfOut = pd.DataFrame()

for x in range(len(coinbase_coins)):
  # print(coinbase_coins.iloc[x, 0])
  # print(coinbase_coins.iloc[x, 0])
  ########################
  # Hourly data
  ########################
  ## tmp = (yf.download(tickers=(str(coinbase_coins.iloc[x, 0]) + '-' +   str('USD')), period = '72h', interval = '1h', progress=False))
  ## tmp = tmp[tmp['Volume'] != 0].tail(1)
  ########################
  # Daily Data
  ########################
  tmp = (yf.download(tickers=(str(coinbase_coins.iloc[x, 0]) + '-' +   str('USD')), period = '180d', interval = '1d', progress=False))
  tmp = tmp[tmp['Volume'] != 0]
  # print(tmp.shape)
  tmp['Coin'] = coinbase_coins.iloc[x, 0]

  if (tmp.shape[0] != 0):
    print(coinbase_coins.iloc[x, 0])
    dfOut = dfOut.append(tmp)
    print(dfOut.shape)
  # dfOut = dfOut.append(tmp)


1 Failed download:
- 1INCH-USD: No data found, symbol may be delisted
AAVE
(74, 7)
ADA
(254, 7)
ALGO
(434, 7)
ANKR
(508, 7)

1 Failed download:
- ATOM-USD: No data found, symbol may be delisted
BAL
(534, 7)
BAND
(608, 7)
BAT
(788, 7)
BCH
(968, 7)
BNT
(1148, 7)
BSV
(1328, 7)
BTC
(1508, 7)

1 Failed download:
- CGLD-USD: No data found, symbol may be delisted
COMP
(1582, 7)
CRV
(1656, 7)

1 Failed download:
- CTSI-USD: No data found, symbol may be delisted
CVC
(1836, 7)

1 Failed download:
- DAI-USD: No data found, symbol may be delisted
DASH
(2016, 7)
DNT
(2196, 7)
ENJ
(2270, 7)
EOS
(2450, 7)
ETC
(2630, 7)
ETH
(2810, 7)

1 Failed download:
- FORTH-USD: No data found, symbol may be delisted
FIL
(2884, 7)

1 Failed download:
- GRT-USD: No data found, symbol may be delisted
GNT
(2907, 7)

1 Failed download:
- ICP-USD: No data found, symbol may be delisted
KNC
(3087, 7)
LINK
(3267, 7)

1 Failed download:
- LOOM-USD: No data found, symbol may be delisted
LRC
(3447, 7)
LTC
(3627, 7)
MANA
(370

In [ ]:
dfOut.tail()

,Open,High,Low,Close,Adj Close,Volume,Coin
Date,,,,,,,
2021-06-09,0.326423,0.353762,0.313131,0.343331,0.343331,3154332929,DOGE
2021-06-10,0.343339,0.344529,0.323278,0.326351,0.326351,2114010549,DOGE
2021-06-11,0.326364,0.333223,0.311502,0.319359,0.319359,1834446577,DOGE
2021-06-12,0.318893,0.319332,0.301804,0.312084,0.312084,1605633791,DOGE
2021-06-13,0.310998,0.315415,0.308291,0.310851,0.310851,1212200320,DOGE


In [ ]:
dfOut[dfOut['Coin'] == 'ADA'].tail()

,Open,High,Low,Close,Adj Close,Volume,Coin
Date,,,,,,,
2021-06-09,1.583486,1.630414,1.485165,1.624826,1.624826,3551319865,ADA
2021-06-10,1.627778,1.635973,1.508578,1.531857,1.531857,2366588153,ADA
2021-06-11,1.533137,1.546059,1.420852,1.436378,1.436378,2477031735,ADA
2021-06-12,1.437721,1.493530,1.358179,1.480220,1.480220,2801038421,ADA
2021-06-13,1.476252,1.501336,1.435600,1.437139,1.437139,2304746240,ADA


In [ ]:
dfOut = dfOut.reset_index()

In [ ]:
dfOut.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Coin
6808,2021-06-09,0.326423,0.353762,0.313131,0.343331,0.343331,3154332929,DOGE
6809,2021-06-10,0.343339,0.344529,0.323278,0.326351,0.326351,2114010549,DOGE
6810,2021-06-11,0.326364,0.333223,0.311502,0.319359,0.319359,1834446577,DOGE
6811,2021-06-12,0.318893,0.319332,0.301804,0.312084,0.312084,1605633791,DOGE
6812,2021-06-13,0.310998,0.315415,0.308291,0.310851,0.310851,1212200320,DOGE


In [ ]:
dfOut['Coin'].unique()

array(['AAVE', 'ADA', 'ALGO', 'ANKR', 'BAL', 'BAND', 'BAT', 'BCH', 'BNT',
       'BSV', 'BTC', 'COMP', 'CRV', 'CVC', 'DASH', 'DNT', 'ENJ', 'EOS',
       'ETC', 'ETH', 'FIL', 'GNT', 'KNC', 'LINK', 'LRC', 'LTC', 'MANA',
       'MATIC', 'MIR', 'MKR', 'NKN', 'NU', 'OMG', 'OXT', 'REP', 'RLC',
       'SUSHI', 'SNX', 'STORJ', 'USDC', 'USDT', 'UMA', 'XLM', 'XRP',
       'XTZ', 'YFI', 'ZEC', 'ZRX', 'DOGE'], dtype=object)

In [ ]:
dfOut[dfOut['Coin'] == 'ADA'].tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Coin
249,2021-06-09,1.583486,1.630414,1.485165,1.624826,1.624826,3551319865,ADA
250,2021-06-10,1.627778,1.635973,1.508578,1.531857,1.531857,2366588153,ADA
251,2021-06-11,1.533137,1.546059,1.420852,1.436378,1.436378,2477031735,ADA
252,2021-06-12,1.437721,1.493530,1.358179,1.480220,1.480220,2801038421,ADA
253,2021-06-13,1.476252,1.501336,1.435600,1.437139,1.437139,2304746240,ADA


In [ ]:
dfOut.loc[:,'Coin'].head()

0    AAVE
1    AAVE
2    AAVE
3    AAVE
4    AAVE
Name: Coin, dtype: object

In [ ]:
# Reference Date

datetime.now() - timedelta(days = (120 + 0))

datetime.datetime(2021, 2, 13, 11, 56, 37, 223930)

In [ ]:
# Max Date

datetime.now() - timedelta(days = 0)


datetime.datetime(2021, 6, 13, 11, 56, 37, 235028)

In [ ]:
# Min Date

datetime.now() - timedelta(days = (0 - 1))


datetime.datetime(2021, 6, 14, 11, 56, 37, 246305)

In [ ]:
###################################################
# Need logic to handle date changes in data
###################################################

dfOut[dfOut['Coin'] == 'ADA']['Date'].max() > date.today()



False

In [ ]:
# dfOut[dfOut['Coin'] == 'ADA'][(dfOut[dfOut['Coin'] == 'ADA']['Date'] >= (datetime.now() - timedelta(days = (120 + 0)))) & (dfOut[dfOut['Coin'] == 'ADA']['Date'] < (datetime.now() - timedelta(days = 0)))]

dfOut[dfOut['Coin'] == 'ADA']

,Date,Open,High,Low,Close,Adj Close,Volume,Coin
74,2020-12-16,0.154312,0.167599,0.149976,0.167589,0.167589,976699799,ADA
75,2020-12-17,0.167571,0.174342,0.161644,0.163483,0.163483,1188727265,ADA
76,2020-12-18,0.163449,0.168635,0.159152,0.164988,0.164988,1101018945,ADA
77,2020-12-19,0.164980,0.170486,0.163320,0.164714,0.164714,586684841,ADA
78,2020-12-20,0.164728,0.167724,0.158695,0.161988,0.161988,729191687,ADA
...,...,...,...,...,...,...,...,...
249,2021-06-09,1.583486,1.630414,1.485165,1.624826,1.624826,3551319865,ADA
250,2021-06-10,1.627778,1.635973,1.508578,1.531857,1.531857,2366588153,ADA
251,2021-06-11,1.533137,1.546059,1.420852,1.436378,1.436378,2477031735,ADA
252,2021-06-12,1.437721,1.493530,1.358179,1.480220,1.480220,2801038421,ADA


In [ ]:
(datetime.today() - timedelta(days = 1))

datetime.datetime(2021, 6, 12, 11, 56, 37, 317110)

In [ ]:
#############################
# address time difference 
#############################

## dfOut[dfOut['Date'] > (datetime.today() - timedelta(hours = 5))]['Date'] = (datetime.today() - timedelta(days = 1))


In [ ]:
dfOut[dfOut['Date'] > (datetime.today() - timedelta(hours = 5))]['Date']

Series([], Name: Date, dtype: datetime64[ns])

In [ ]:
dfOut[(dfOut['Coin'].isin(['BTC', 'ADA', 'MATIC', 'STORJ', 'XLM']))]

,Date,Open,High,Low,Close,Adj Close,Volume,Coin
74,2020-12-16,0.154312,0.167599,0.149976,0.167589,0.167589,976699799,ADA
75,2020-12-17,0.167571,0.174342,0.161644,0.163483,0.163483,1188727265,ADA
76,2020-12-18,0.163449,0.168635,0.159152,0.164988,0.164988,1101018945,ADA
77,2020-12-19,0.164980,0.170486,0.163320,0.164714,0.164714,586684841,ADA
78,2020-12-20,0.164728,0.167724,0.158695,0.161988,0.161988,729191687,ADA
...,...,...,...,...,...,...,...,...
5834,2021-06-09,0.349703,0.364540,0.325290,0.364291,0.364291,914569105,XLM
5835,2021-06-10,0.365613,0.367092,0.332172,0.339210,0.339210,822163707,XLM
5836,2021-06-11,0.339390,0.343038,0.315225,0.318661,0.318661,748666312,XLM
5837,2021-06-12,0.318603,0.343686,0.300206,0.328297,0.328297,848257045,XLM


In [ ]:
# dfOut[dfOut['Date'] > datetime.today().strftime('%Y-%m-%d')]
datetime.today().strftime('%Y-%m-%d')

'2021-06-13'

In [ ]:
dfOut[dfOut['Date'] == dfOut['Date'].max()]

,Date,Open,High,Low,Close,Adj Close,Volume,Coin
73,2021-06-13,279.439484,286.773468,267.801270,284.410004,284.410004,232382256,AAVE
253,2021-06-13,1.476252,1.501336,1.435600,1.437139,1.437139,2304746240,ADA
433,2021-06-13,0.990391,1.021714,0.965841,1.016532,1.016532,119982400,ALGO
507,2021-06-13,0.084075,0.084147,0.078944,0.082432,0.082432,34226112,ANKR
533,2021-06-13,22.426500,23.073683,21.915859,22.314688,22.314688,21535176,BAL
607,2021-06-13,6.499582,6.688992,6.272043,6.424265,6.424265,35158236,BAND
787,2021-06-13,0.617505,0.629166,0.602607,0.612722,0.612722,115193968,BAT
967,2021-06-13,580.098999,587.136475,561.006104,569.782288,569.782288,1524348544,BCH
1147,2021-06-13,3.747997,3.811394,3.651947,3.704611,3.704611,47075404,BNT
1327,2021-06-13,162.087097,165.576508,159.911728,162.109818,162.109818,588574912,BSV


# Process AI


In [ ]:
###########################
# assume long data
###########################

def get_sim(inputData, histDays, backcastDays, numLags, forecastDays):

  resultsOut = pd.DataFrame()
  tmp = []

###########################
# Loop over backcast days
###########################

  for day in range(backcastDays):
    print(day)

###########################
# partition data by last X days 
# and previous Y days
###########################

    ref_date = datetime.now() - timedelta(days = (histDays + day))
    print("Reference Date:" )
    print(ref_date)

    max_date = datetime.now() - timedelta(days = day)
    print("Max Date:")
    print(max_date)

###############################
#
# IMPORTANT!!
# 
###############################

    fcst_date = datetime.now() - timedelta(days = (day - 1))
    ## fcst_date = datetime.now() - timedelta(days = (day))
    print("Forecast Date:")
    print(fcst_date)
    # max_date = pd.Timestamp('now').floor('D') 
    

# + pd.Timedelta(-1, unit='D')

#################################
# Take last Z days
# loop over backCast days
#################################

    tmp = inputData[(inputData['Date'] >= ref_date) & (inputData['Date'] < max_date)]
    print("Observed Min Date:")
    print(tmp['Date'].min())
    print("Observed Max Date:")
    print(tmp['Date'].max())

    tmp = tmp.set_index(['Date'])

#################################
# Loop over Coins
#################################

    for coin in inputData['Coin'].unique():

      print(coin)

###########################
# ETL long data to wide 
# add Log(Volume)
###########################

      mytmp = tmp[tmp['Coin'] == coin]

      # yf.download(tickers='DOGE-USD', period = '90d', interval = '1d')

      print(mytmp.head(), mytmp.tail())

      myBTC = tmp[tmp['Coin'] == 'BTC']
      myBTC.columns = ('BTC-' + myBTC.columns)

      mytmp2 = mytmp.merge(myBTC, how = 'left', on='Date')

      mytmp2['logVolume'] = np.log(mytmp2.loc[:, 'Volume'])
      mytmp2['BTC-logVolume'] = np.log(mytmp2.loc[:, 'BTC-Volume'])

      # print(mytmp2)

      # yf.download(tickers='BTC-USD', period = '90d', interval = '1d')

      # tmp1 = tmp[tmp['Coin'] == coin]  

      # tmp2 = tmp1.pivot_table(index=['Date', 'Coin'], columns=['Variable'], values='Value').reset_index()

      # mytmp2 = mytmp2.set_index(['Date'])

      # tmp2 = tmp2.sort_values(['Coin', 'Date'], ascending = True)

      # tmp2['logVolume'] = np.log(tmp2.loc[:,'5. volume'])

      ###########################
      # Create Daily High-Low Diff
      ###########################
      
      mytmp2['dailyHighLow'] = mytmp2['High'] - mytmp2['Low']

      # mytmp2_use = mytmp2[['Open', 'High', 'Low', 'Close', 'Adj Close', 'BTC-Open', 'BTC-High', 'BTC-Low', 'BTC-Close', 'BTC-Adj Close', 'logVolume', 'BTC-logVolume', 'dailyHighLow']]
      mytmp2_use = mytmp2[['Open', 'High', 'Low', 'Close', 'logVolume', 'dailyHighLow', 'BTC-Open', 'BTC-logVolume']]


      # print((mytmp2_use.shape))

      if mytmp2_use.shape[0] > 9:
      
      # tmp2_out = tmp2[['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', '4b. close (USD)', 'logVolume', 'Coin']]

      #########################
      # Remove Closing Price.  Lag1Close = Lag0Open
      #########################

      # tmp2_use = tmp2[['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', 'logVolume', 'dailyHighLow']]

      # tmp2_out = tmp2[['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', 'logVolume', 'dailyHighLow', 'Coin']]

      # print(tmp2_out.tail())

      # print(tmp['Date'].max())

      # print(mytmp2_use.head(), mytmp2_use.tail())

  ###################################
  # compute changes
  ###################################

        data_ret = (mytmp2_use).diff(1).dropna()

        # data_ret = data_ret.set_index(['Date'])

        # print(data_ret.head())

        # print(data_ret.head(), data_ret.tail())

        # print(data_ret.info(verbose=True))

  ###################################
  # construct model
  ###################################

        model = VAR(data_ret, freq='D')

        # print(data_ret)

  ###################################
  # Fit model using 8 lags
  ###################################

        results = model.fit(numLags)

      # print(results.summary())

  ###########################
  # append forecast
  ###########################

        mytmp2_use_fcst = pd.DataFrame(results.forecast(data_ret.values[-numLags:], forecastDays))

        mytmp2_use_fcst.columns = mytmp2_use.columns

        mytmp2_use_fcst['Coin'] = coin

        mytmp2_use_fcst['Date'] = (fcst_date).strftime('%Y-%m-%d')
        # print((datetime.datetime.now() + datetime.timedelta(day)).strftime('%Y-%m-%d'))
        mytmp2_use_fcst = mytmp2_use_fcst.set_index(['Date'])

        # tmp2_out = data_ret.append(tmp2_use_fcst, ignore_index = False)

        # print(tmp2_use_fcst)

        resultsOut = resultsOut.append(mytmp2_use_fcst, ignore_index = False)

    #     # print(resultsOut)

    # resultsOut.columns = ['TS', 'P', 'Variable', 'Coin', 'prev30dayAvg', 'prior30dayAvg']
    
  return resultsOut


In [ ]:
###################################
# NOTE: SOME TICKERS BREAKING VAR, OTHER TIMES THEY DO NOT
# DATA IS INCONSISTENT DUE TO UTC v. CST
##################################

# myfcasts = get_sim(inputData = dfOut[-dfOut['Coin'].isin(['AAVE', 'CRV','NU', 'SUSHI'])], histDays = 120, backcastDays = 30, numLags = 5, forecastDays = 1)

myfcasts = get_sim(inputData = dfOut[(dfOut['Coin'].isin(['ADA', 'BTC', 'AAVE', 'MATIC', 'STORJ', 'XLM']))], histDays = 90, backcastDays = 30, numLags = 3, forecastDays = 1)

#  & (dfOut['Date'] < '2021-06-12')

0
Reference Date:
2021-03-15 11:56:37.667818
Max Date:
2021-06-13 11:56:37.668245
Forecast Date:
2021-06-14 11:56:37.668630
Observed Min Date:
2021-03-16 00:00:00
Observed Max Date:
2021-06-13 00:00:00
AAVE
                  Open        High         Low  ...   Adj Close     Volume  Coin
Date                                            ...                             
2021-04-01  377.183685  385.298737  369.995331  ...  377.611786  342371946  AAVE
2021-04-02  377.514648  408.479919  374.290924  ...  406.692291  480081712  AAVE
2021-04-03  406.883606  411.377075  385.415894  ...  385.420563  395544800  AAVE
2021-04-04  385.526978  409.579529  382.788849  ...  404.309723  330959728  AAVE
2021-04-05  404.392944  407.329315  388.300354  ...  397.924561  306918909  AAVE

[5 rows x 7 columns]                   Open        High         Low  ...   Adj Close     Volume  Coin
Date                                            ...                             
2021-06-09  330.009796  342.554352  306.53

In [ ]:
myfcasts[myfcasts['Coin'] == 'ADA'].sort_values(by=['Coin', 'Date']).tail()

,Open,High,Low,Close,logVolume,dailyHighLow,BTC-Open,BTC-logVolume,Coin
Date,,,,,,,,,
2021-06-10,0.041407,0.168191,0.123612,0.129045,0.202823,0.044579,1259.744968,-0.239384,ADA
2021-06-11,-0.093267,0.028407,-0.130902,0.094551,0.292431,0.159309,-1425.297496,0.159833,ADA
2021-06-12,-0.094913,-0.112064,-0.082216,-0.016304,-0.002079,-0.029849,-2834.316598,0.132138,ADA
2021-06-13,0.041745,-0.030291,0.096767,-0.029773,-0.345211,-0.127057,1816.814626,-0.009522,ADA
2021-06-14,-0.039320,0.023025,0.074581,0.087112,0.057704,-0.051556,26.262650,0.085865,ADA


In [ ]:
dfOut.set_index('Date', inplace=True)

In [ ]:
dfOut[dfOut['Coin'] == 'ADA'].tail()

,Open,High,Low,Close,Adj Close,Volume,Coin
Date,,,,,,,
2021-06-09,1.583486,1.630414,1.485165,1.624826,1.624826,3551319865,ADA
2021-06-10,1.627778,1.635973,1.508578,1.531857,1.531857,2366588153,ADA
2021-06-11,1.533137,1.546059,1.420852,1.436378,1.436378,2477031735,ADA
2021-06-12,1.437721,1.493530,1.358179,1.480220,1.480220,2801038421,ADA
2021-06-13,1.476252,1.501336,1.435600,1.437139,1.437139,2304746240,ADA


In [ ]:
myfcasts[myfcasts['Coin'] == 'ADA'].head()

,Open,High,Low,Close,logVolume,dailyHighLow,BTC-Open,BTC-logVolume,Coin
Date,,,,,,,,,
2021-06-14,-0.039320,0.023025,0.074581,0.087112,0.057704,-0.051556,26.262650,0.085865,ADA
2021-06-13,0.041745,-0.030291,0.096767,-0.029773,-0.345211,-0.127057,1816.814626,-0.009522,ADA
2021-06-12,-0.094913,-0.112064,-0.082216,-0.016304,-0.002079,-0.029849,-2834.316598,0.132138,ADA
2021-06-11,-0.093267,0.028407,-0.130902,0.094551,0.292431,0.159309,-1425.297496,0.159833,ADA
2021-06-10,0.041407,0.168191,0.123612,0.129045,0.202823,0.044579,1259.744968,-0.239384,ADA


In [ ]:
# data_in_long['nextDate'] = 
# myfcasts.reset_index(inplace=True)
# (myfcasts['Date'] - timedelta(days = (-1)))

In [ ]:
myfcasts.reset_index(inplace=True)

In [ ]:
myfcasts['Date'] = pd.to_datetime(myfcasts['Date'])

In [ ]:
myfcasts.head()

,Date,Open,High,Low,Close,logVolume,dailyHighLow,BTC-Open,BTC-logVolume,Coin
0,2021-06-14,5.675270,14.465975,11.274529,20.853586,0.177722,3.191446,567.165723,-0.027810,AAVE
1,2021-06-14,-0.030484,0.028959,0.083123,0.083840,0.048428,-0.054164,174.729802,0.079258,ADA
2,2021-06-14,412.822419,925.976326,-724.766844,-420.984593,0.085046,1650.743171,412.822419,0.085046,BTC
3,2021-06-14,0.036495,-0.030324,0.103559,0.078793,-0.076095,-0.133883,757.646107,0.023777,MATIC
4,2021-06-14,0.017360,-0.045270,-0.038165,-0.079552,0.070313,-0.007105,-442.317435,0.051997,STORJ


In [ ]:
##############################
# IMPORTANT!!
# reset Forecast Date to Today
##############################

## myfcasts['priorDate'] = (myfcasts['Date'] - timedelta(days = (1)))
myfcasts['priorDate'] = (myfcasts['Date'] - timedelta(days = (0)))


In [ ]:
myfcasts.head()

,Date,Open,High,Low,Close,logVolume,dailyHighLow,BTC-Open,BTC-logVolume,Coin,priorDate
0,2021-06-14,5.675270,14.465975,11.274529,20.853586,0.177722,3.191446,567.165723,-0.027810,AAVE,2021-06-14
1,2021-06-14,-0.030484,0.028959,0.083123,0.083840,0.048428,-0.054164,174.729802,0.079258,ADA,2021-06-14
2,2021-06-14,412.822419,925.976326,-724.766844,-420.984593,0.085046,1650.743171,412.822419,0.085046,BTC,2021-06-14
3,2021-06-14,0.036495,-0.030324,0.103559,0.078793,-0.076095,-0.133883,757.646107,0.023777,MATIC,2021-06-14
4,2021-06-14,0.017360,-0.045270,-0.038165,-0.079552,0.070313,-0.007105,-442.317435,0.051997,STORJ,2021-06-14


In [ ]:
myfcasts.rename(columns={"Date": "Forecast Date", "priorDate": "Date"}, inplace=True)


In [ ]:
myfcasts.set_index('Date', inplace=True)


In [ ]:
##############################
# the fcasts data output  
##############################

myfcasts.head()


,Forecast Date,Open,High,Low,Close,logVolume,dailyHighLow,BTC-Open,BTC-logVolume,Coin
Date,,,,,,,,,,
2021-06-14,2021-06-14,5.675270,14.465975,11.274529,20.853586,0.177722,3.191446,567.165723,-0.027810,AAVE
2021-06-14,2021-06-14,-0.030484,0.028959,0.083123,0.083840,0.048428,-0.054164,174.729802,0.079258,ADA
2021-06-14,2021-06-14,412.822419,925.976326,-724.766844,-420.984593,0.085046,1650.743171,412.822419,0.085046,BTC
2021-06-14,2021-06-14,0.036495,-0.030324,0.103559,0.078793,-0.076095,-0.133883,757.646107,0.023777,MATIC
2021-06-14,2021-06-14,0.017360,-0.045270,-0.038165,-0.079552,0.070313,-0.007105,-442.317435,0.051997,STORJ


In [ ]:
##############################
# go back to original data.  
##############################

dfOut.tail()


,Open,High,Low,Close,Adj Close,Volume,Coin
Date,,,,,,,
2021-06-09,0.326423,0.353762,0.313131,0.343331,0.343331,3154332929,DOGE
2021-06-10,0.343339,0.344529,0.323278,0.326351,0.326351,2114010549,DOGE
2021-06-11,0.326364,0.333223,0.311502,0.319359,0.319359,1834446577,DOGE
2021-06-12,0.318893,0.319332,0.301804,0.312084,0.312084,1605633791,DOGE
2021-06-13,0.310998,0.315415,0.308291,0.311533,0.311533,1210525824,DOGE


In [ ]:
dfOut.dtypes


Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
Coin          object
dtype: object

In [ ]:
dfOut.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6813 entries, 2021-04-01 to 2021-06-13
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       6813 non-null   float64
 1   High       6813 non-null   float64
 2   Low        6813 non-null   float64
 3   Close      6813 non-null   float64
 4   Adj Close  6813 non-null   float64
 5   Volume     6813 non-null   int64  
 6   Coin       6813 non-null   object 
dtypes: float64(5), int64(1), object(1)
memory usage: 425.8+ KB


In [ ]:
myfcasts.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 180 entries, 2021-06-14 to 2021-05-16
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Forecast Date  180 non-null    datetime64[ns]
 1   Open           180 non-null    float64       
 2   High           180 non-null    float64       
 3   Low            180 non-null    float64       
 4   Close          180 non-null    float64       
 5   logVolume      180 non-null    float64       
 6   dailyHighLow   180 non-null    float64       
 7   BTC-Open       180 non-null    float64       
 8   BTC-logVolume  180 non-null    float64       
 9   Coin           180 non-null    object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 15.5+ KB


In [ ]:
# dfOut['Coin']

In [ ]:
dfOut.columns = ('Actual: ' + dfOut.columns)

In [ ]:
dfOut.rename(columns={"Actual: Coin": "Coin"}, inplace=True)

In [ ]:
dfOut[dfOut['Coin'] == 'ADA'].tail(20)

,Actual: Open,Actual: High,Actual: Low,Actual: Close,Actual: Adj Close,Actual: Volume,Coin
Date,,,,,,,
2021-05-25,1.550373,1.648563,1.439744,1.555369,1.555369,5723130829,ADA
2021-05-26,1.554463,1.832433,1.554463,1.782702,1.782702,6638876193,ADA
2021-05-27,1.781137,1.790477,1.612414,1.645960,1.645960,4266751905,ADA
2021-05-28,1.650636,1.677707,1.452245,1.511987,1.511987,5373114660,ADA
2021-05-29,1.513623,1.561741,1.335859,1.406232,1.406232,3902596499,ADA
2021-05-30,1.404303,1.688977,1.351820,1.577279,1.577279,4974774760,ADA
2021-05-31,1.575644,1.750620,1.527209,1.742778,1.742778,4491096277,ADA
2021-06-01,1.738447,1.779591,1.660549,1.734753,1.734753,4123980990,ADA
2021-06-02,1.736103,1.803849,1.719255,1.752824,1.752824,3453620094,ADA


In [ ]:
final_output = myfcasts.merge(dfOut, how='left', on=('Date', 'Coin'), )


In [ ]:
###############################
# QA this mudder 
###############################

final_output[final_output['Coin'] == 'ADA'].head(10)


,Forecast Date,Open,High,Low,Close,logVolume,dailyHighLow,BTC-Open,BTC-logVolume,Coin,Actual: Open,Actual: High,Actual: Low,Actual: Close,Actual: Adj Close,Actual: Volume
Date,,,,,,,,,,,,,,,,
2021-06-14,2021-06-14,-0.030484,0.028959,0.083123,0.083840,0.048428,-0.054164,174.729802,0.079258,ADA,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-13,2021-06-13,0.041745,-0.030291,0.096767,-0.029773,-0.345211,-0.127057,1816.814626,-0.009522,ADA,1.476252,1.501336,1.435600,1.446149,1.446149,2.326643e+09
2021-06-12,2021-06-12,-0.094913,-0.112064,-0.082216,-0.016304,-0.002079,-0.029849,-2834.316598,0.132138,ADA,1.437721,1.493530,1.358179,1.480220,1.480220,2.801038e+09
2021-06-11,2021-06-11,-0.093267,0.028407,-0.130902,0.094551,0.292431,0.159309,-1425.297496,0.159833,ADA,1.533137,1.546059,1.420852,1.436378,1.436378,2.477032e+09
2021-06-10,2021-06-10,0.041407,0.168191,0.123612,0.129045,0.202823,0.044579,1259.744968,-0.239384,ADA,1.627778,1.635973,1.508578,1.531857,1.531857,2.366588e+09
2021-06-09,2021-06-09,0.018344,0.008575,0.014518,-0.123193,-0.227202,-0.005943,-275.088484,-0.265630,ADA,1.583486,1.630414,1.485165,1.624826,1.624826,3.551320e+09
2021-06-08,2021-06-08,-0.112681,-0.058840,0.016355,0.086174,0.021101,-0.075195,-2570.484916,-0.050080,ADA,1.567051,1.594298,1.402785,1.588030,1.588030,4.318242e+09
2021-06-07,2021-06-07,0.025919,0.032707,-0.045879,-0.017876,0.272969,0.078586,881.409079,0.135905,ADA,1.677226,1.740569,1.557106,1.564106,1.564106,2.644320e+09
2021-06-06,2021-06-06,-0.046651,-0.078765,-0.123408,-0.051391,0.040938,0.044642,380.131561,0.081953,ADA,1.656461,1.706524,1.647702,1.678248,1.678248,1.748046e+09


In [ ]:
final_output.reset_index(inplace=True)


In [ ]:
final_output.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               180 non-null    datetime64[ns]
 1   Forecast Date      180 non-null    datetime64[ns]
 2   Open               180 non-null    float64       
 3   High               180 non-null    float64       
 4   Low                180 non-null    float64       
 5   Close              180 non-null    float64       
 6   logVolume          180 non-null    float64       
 7   dailyHighLow       180 non-null    float64       
 8   BTC-Open           180 non-null    float64       
 9   BTC-logVolume      180 non-null    float64       
 10  Coin               180 non-null    object        
 11  Actual: Open       174 non-null    float64       
 12  Actual: High       174 non-null    float64       
 13  Actual: Low        174 non-null    float64       
 14  Actual: Cl

In [ ]:
####################
# format column names
####################

final_output.columns = final_output.columns.str.replace('[#,@,&,:, ,-]', '_')


In [ ]:
final_output.columns

Index(['Date', 'Forecast_Date', 'Open', 'High', 'Low', 'Close', 'logVolume',
       'dailyHighLow', 'BTC_Open', 'BTC_logVolume', 'Coin', 'Actual__Open',
       'Actual__High', 'Actual__Low', 'Actual__Close', 'Actual__Adj_Close',
       'Actual__Volume'],
      dtype='object')

In [ ]:
##############################
# paste previous day price
##############################

dfOut[dfOut['Coin'] == 'ADA'].sort_values(by = 'Date', ascending = False).head(10)


,Actual: Open,Actual: High,Actual: Low,Actual: Close,Actual: Adj Close,Actual: Volume,Coin
Date,,,,,,,
2021-06-13,1.476252,1.501336,1.435600,1.446149,1.446149,2326642688,ADA
2021-06-12,1.437721,1.493530,1.358179,1.480220,1.480220,2801038421,ADA
2021-06-11,1.533137,1.546059,1.420852,1.436378,1.436378,2477031735,ADA
2021-06-10,1.627778,1.635973,1.508578,1.531857,1.531857,2366588153,ADA
2021-06-09,1.583486,1.630414,1.485165,1.624826,1.624826,3551319865,ADA
2021-06-08,1.567051,1.594298,1.402785,1.588030,1.588030,4318242183,ADA
2021-06-07,1.677226,1.740569,1.557106,1.564106,1.564106,2644319967,ADA
2021-06-06,1.656461,1.706524,1.647702,1.678248,1.678248,1748045655,ADA
2021-06-05,1.709916,1.783177,1.611715,1.659009,1.659009,2969362191,ADA


In [ ]:
tmpdfOut = dfOut.reset_index()

In [ ]:
#########################################
# IMPORTANT
# Another date adjustment 
#########################################

## tmpdfOut['priorDate'] = (tmpdfOut['Date'] - timedelta(days = (1)))
tmpdfOut['priorDate'] = (tmpdfOut['Date'] - timedelta(days = (0)))


In [ ]:
tmpdfOut.sort_values(by = ['Coin', 'priorDate'], ascending = False)

,Date,Actual: Open,Actual: High,Actual: Low,Actual: Close,Actual: Adj Close,Actual: Volume,Coin,priorDate
6632,2021-06-13,0.848101,0.865651,0.820322,0.833249,0.833249,59405828,ZRX,2021-06-13
6631,2021-06-12,0.863818,0.890655,0.804930,0.851176,0.851176,67316703,ZRX,2021-06-12
6630,2021-06-11,0.932439,0.949485,0.852698,0.863741,0.863741,60248504,ZRX,2021-06-11
6629,2021-06-10,0.966212,0.983284,0.913637,0.932484,0.932484,66980060,ZRX,2021-06-10
6628,2021-06-09,0.955664,0.980978,0.926087,0.964683,0.964683,118768929,ZRX,2021-06-09
...,...,...,...,...,...,...,...,...,...
4,2021-04-05,404.392944,407.329315,388.300354,397.924561,397.924561,306918909,AAVE,2021-04-05
3,2021-04-04,385.526978,409.579529,382.788849,404.309723,404.309723,330959728,AAVE,2021-04-04
2,2021-04-03,406.883606,411.377075,385.415894,385.420563,385.420563,395544800,AAVE,2021-04-03
1,2021-04-02,377.514648,408.479919,374.290924,406.692291,406.692291,480081712,AAVE,2021-04-02


In [ ]:
tmpdfOut_lag1 = tmpdfOut[['priorDate', 'Coin', 'Actual: Close']]

In [ ]:
tmpdfOut_lag1[tmpdfOut_lag1['Coin'] == 'DASH'].tail(10)

,priorDate,Coin,Actual: Close
2006,2021-06-04,DASH,191.689560
2007,2021-06-05,DASH,185.272354
2008,2021-06-06,DASH,185.738083
2009,2021-06-07,DASH,171.609970
2010,2021-06-08,DASH,166.783798
2011,2021-06-09,DASH,176.426849
2012,2021-06-10,DASH,169.814270
2013,2021-06-11,DASH,165.677200
2014,2021-06-12,DASH,163.696426
2015,2021-06-13,DASH,159.452682


In [ ]:
tmpdfOut_lag1.rename(columns={"priorDate": "Date", "Actual: Close": "Tomorrow_Close", }, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
tmpdfOut_lag1[tmpdfOut_lag1['Coin'] == 'DASH'].tail()

,Date,Coin,Tomorrow_Close
2011,2021-06-09,DASH,176.426849
2012,2021-06-10,DASH,169.814270
2013,2021-06-11,DASH,165.677200
2014,2021-06-12,DASH,163.696426
2015,2021-06-13,DASH,159.452682


In [ ]:
final_output[final_output['Coin'] == 'DASH'].head()

,Date,Forecast_Date,Open,High,Low,Close,logVolume,dailyHighLow,BTC_Open,BTC_logVolume,Coin,Actual__Open,Actual__High,Actual__Low,Actual__Close,Actual__Adj_Close,Actual__Volume


In [ ]:
##############################
# extract only necessary data
##############################

tmpfcast = final_output[['Coin', 'Date', 'Actual__Close', 'Forecast_Date', 'Close']]


In [ ]:
tmpfcast[tmpfcast['Coin'] == 'DASH'].head()

,Coin,Date,Actual__Close,Forecast_Date,Close


In [ ]:

tmpfcast['Close Price'] = tmpfcast['Actual__Close'] + tmpfcast['Close']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# WRONG
# tmpfcast['Miss'] = tmpfcast['Close Price'] - tmpfcast['Actual__Close']

# .sort_values(['Coin', 'Date'], ascending = False).head(30)

tmpfcast.sort_values(['Coin', 'Date'], ascending = False).head(10)


,Coin,Date,Actual__Close,Forecast_Date,Close,Close Price
5,XLM,2021-06-14,NaN,2021-06-14,-0.036696,NaN
11,XLM,2021-06-13,0.323510,2021-06-13,0.006913,0.330422
17,XLM,2021-06-12,0.328297,2021-06-12,0.035180,0.363477
23,XLM,2021-06-11,0.318661,2021-06-11,0.019394,0.338055
29,XLM,2021-06-10,0.339210,2021-06-10,-0.005503,0.333707
35,XLM,2021-06-09,0.364291,2021-06-09,-0.025090,0.339201
41,XLM,2021-06-08,0.350567,2021-06-08,0.018157,0.368724
47,XLM,2021-06-07,0.346232,2021-06-07,-0.001905,0.344327
53,XLM,2021-06-06,0.381255,2021-06-06,-0.010875,0.370380
59,XLM,2021-06-05,0.375380,2021-06-05,0.004816,0.380196


In [ ]:
tmpfcast[tmpfcast['Coin'] == 'DASH'].head()


,Coin,Date,Actual__Close,Forecast_Date,Close,Close Price


In [ ]:
tmpdfOut_lag1[tmpdfOut_lag1['Coin'] == 'DASH'].tail()

,Date,Coin,Tomorrow_Close
2011,2021-06-09,DASH,176.426849
2012,2021-06-10,DASH,169.814270
2013,2021-06-11,DASH,165.677200
2014,2021-06-12,DASH,163.696426
2015,2021-06-13,DASH,159.452682


In [ ]:
#######################################################
#######################################################


In [ ]:
tmpfcast2 = tmpfcast.merge(tmpdfOut_lag1, how='left', on=['Date', 'Coin']).sort_values(by=['Coin', 'Date'], ascending = True)

In [ ]:
tmpfcast2[tmpfcast2['Coin'] == 'DASH'].tail()

,Coin,Date,Actual__Close,Forecast_Date,Close,Close Price,Tomorrow_Close


In [ ]:
tmpfcast2.rename(columns={"Actual__Close": "Today_Price", "Close": "Forecast_PM", "Close Price": "Forecast_Price", "Tomorrow_Close": "Actual_Price"}, inplace=True)


In [ ]:
tmpfcast2.head()

,Coin,Date,Today_Price,Forecast_Date,Forecast_PM,Forecast_Price,Actual_Price
174,AAVE,2021-05-16,526.214722,2021-05-16,-138.227569,387.987153,526.214722
168,AAVE,2021-05-17,574.259460,2021-05-17,16.534947,590.794408,574.259460
162,AAVE,2021-05-18,632.266479,2021-05-18,86.872603,719.139082,632.266479
156,AAVE,2021-05-19,418.369324,2021-05-19,23.709813,442.079136,418.369324
150,AAVE,2021-05-20,468.812683,2021-05-20,-12.352014,456.460669,468.812683


In [ ]:
tmpfcast2['Actual_PM'] = tmpfcast2['Actual_Price'] - tmpfcast2['Today_Price']


In [ ]:
tmpfcast2 = tmpfcast2[['Coin', 'Date', 'Today_Price', 'Forecast_Date', 'Forecast_PM',
       'Forecast_Price', 'Actual_PM', 'Actual_Price']]
       

In [ ]:
tmpfcast2[tmpfcast2['Coin'] == 'DASH'].tail(20)

,Coin,Date,Today_Price,Forecast_Date,Forecast_PM,Forecast_Price,Actual_PM,Actual_Price


In [ ]:
tmpfcast2['Miss'] = tmpfcast2['Actual_Price'] - tmpfcast2['Today_Price']



In [ ]:
tmpfcast2['absMiss'] = abs(tmpfcast2['Miss'])


In [ ]:
tmpfcast2[tmpfcast2['Coin'] == 'ADA'].tail(20)

,Coin,Date,Today_Price,Forecast_Date,Forecast_PM,Forecast_Price,Actual_PM,Actual_Price,Miss,absMiss
115,ADA,2021-05-26,1.782702,2021-05-26,-0.045530,1.737172,0.0,1.782702,0.0,0.0
109,ADA,2021-05-27,1.645960,2021-05-27,0.123543,1.769503,0.0,1.645960,0.0,0.0
103,ADA,2021-05-28,1.511987,2021-05-28,0.027839,1.539826,0.0,1.511987,0.0,0.0
97,ADA,2021-05-29,1.406232,2021-05-29,-0.068856,1.337376,0.0,1.406232,0.0,0.0
91,ADA,2021-05-30,1.577279,2021-05-30,-0.041944,1.535335,0.0,1.577279,0.0,0.0
85,ADA,2021-05-31,1.742778,2021-05-31,0.051420,1.794198,0.0,1.742778,0.0,0.0
79,ADA,2021-06-01,1.734753,2021-06-01,-0.085808,1.648945,0.0,1.734753,0.0,0.0
73,ADA,2021-06-02,1.752824,2021-06-02,0.026999,1.779823,0.0,1.752824,0.0,0.0
67,ADA,2021-06-03,1.840471,2021-06-03,-0.070302,1.770169,0.0,1.840471,0.0,0.0
61,ADA,2021-06-04,1.710908,2021-06-04,0.059115,1.770023,0.0,1.710908,0.0,0.0


In [ ]:
#################### 
# write to DB
####################

table_id = 'my_dataset.actuals_forecasts'
tmpfcast2.to_gbq(table_id, project_id = 'crypto1-307915', if_exists = 'replace')
print(datetime.now())


1it [00:04,  4.25s/it]

2021-06-13 11:31:18.796617


In [ ]:
# final_output.dtypes


In [ ]:
# final_output['priorDate'] = (final_output['Date'] - timedelta(days = (1)))

In [ ]:
# final_output

In [ ]:
# final_output.dtypes

In [ ]:

# final_output[['Coin', 'priorDate', 'Date', 'Actual__Close', 'Forecast_Date', 'Close']].sort_values(['Coin', 'Date'], ascending = False).head(30)



In [ ]:
######################
# pull a single ticker 
######################

# mytmp = yf.download(tickers='DOGE-USD', period = '48h', interval = '5m')
mytmp = yf.download(tickers='ADA-USD', period = '90d', interval = '1d', tz='CST')

# myBTC = yf.download(tickers='BTC-USD', period = '90d', interval = '1d')


[*********************100%***********************]  1 of 1 completed


In [ ]:
# mytmp = mytmp[mytmp['Volume'] != 0].head()

In [ ]:
# mytmp.shape

In [ ]:
mytmp.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-06-09,1.583486,1.630414,1.485165,1.624826,1.624826,3551319865
2021-06-10,1.627778,1.635973,1.508578,1.531857,1.531857,2366588153
2021-06-11,1.533137,1.546059,1.420852,1.436378,1.436378,2477031735
2021-06-12,1.437721,1.493530,1.358179,1.480220,1.480220,2801038421
2021-06-13,1.476252,1.501336,1.435600,1.446149,1.446149,2326642688


In [ ]:
# myBTC.tail()

In [ ]:
# myBTC.columns = ('BTC-' + myBTC.columns)

In [ ]:
# myBTC.head()

In [ ]:
# mytmp2 = mytmp.merge(myBTC, how = 'left', on='Date')

In [ ]:
# mytmp2.head()

In [ ]:
# mytmp2['BTC-Volume'].plot()
# plt.grid()

In [ ]:
# mytmp2['logVolume'] = np.log(mytmp2.loc[:, 'Volume'])
# mytmp2['BTC-logVolume'] = np.log(mytmp2.loc[:, 'BTC-Volume'])


In [ ]:
# mytmp2.head()

In [ ]:
# mytmp2_use = mytmp2[['Open', 'High', 'Low', 'Adj Close', 'logVolume', 'BTC-Open', 'BTC-High', 'BTC-Low', 'BTC-Adj Close', 'BTC-logVolume']]


In [ ]:
# data_ret = (mytmp2_use).diff(1).dropna()
# data_ret = (mytmp).diff(1).dropna()

In [ ]:
# data_ret.head()

In [ ]:
# data_ret.shape

In [ ]:
#data_ret.reset_index(inplace=True)

In [ ]:
#data_ret.head()

In [ ]:
# data_ret.dtypes

In [ ]:
#data_ret.set_index(['Date'], inplace=True)

In [ ]:
# data_ret['Adj Close'].plot()

In [ ]:
# model = VAR(data_ret)

###################################
# Fit model using 8 lags
###################################

# results = model.fit(10)


In [ ]:
# results.params

In [ ]:
##################################
# create 1 day head forecast 
##################################

# pd.DataFrame(results.forecast(data_ret.values[-10:],1))


In [ ]:
# tmp.pivot_table(index=['Variable', 'Date'], 
#                     columns='Coin', 
#                     values='Value').reset_index().round(3)

In [ ]:
# mytmp[mytmp['Volume'] != 0]

In [ ]:
# dfOut

In [ ]:
# dfOut2 = pd.concat(dfOut)

In [ ]:
# dfOut.shape

In [ ]:
# coinbase_coins.iloc[1,0]

In [ ]:
yf.download(tickers=(str(coinbase_coins.iloc[1,0]) + '-' +   str('USD')), period = '2h', interval = '15m').tail(1)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-06-13 12:30:03+01:00,285.999695,285.999695,285.999695,285.999695,285.999695,0


In [ ]:
# yf.download(tickers=(str('ETC') + '-' +   str('USD')), period = '90d', interval = '1d')